In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

# Trips to parse data

In [31]:
# Open interest BTC from Grey Scal
data = requests.get('https://fapi.bybt.com/api/grayscaleOpenInterest/history?symbol=BTC')
data = data.__dict__
data = json.loads(data.get('_content'))
data = data.get('data')

In [34]:
df = {'timestamp': data.get('dateList'), 'open_interest': data.get('opList'), 'price': data.get('priceList')}
# df = pd.DataFrame(data)
#

In [35]:
df

{'timestamp': [1591660800000,
  1591747200000,
  1591833600000,
  1591920000000,
  1592006400000,
  1592092800000,
  1592179200000,
  1592265600000,
  1592352000000,
  1592438400000,
  1592524800000,
  1592611200000,
  1592697600000,
  1592784000000,
  1592870400000,
  1592956800000,
  1593043200000,
  1593129600000,
  1593216000000,
  1593302400000,
  1593388800000,
  1593475200000,
  1593561600000,
  1593648000000,
  1593734400000,
  1593820800000,
  1593907200000,
  1593993600000,
  1594080000000,
  1594166400000,
  1594252800000,
  1594339200000,
  1594425600000,
  1594512000000,
  1594598400000,
  1594684800000,
  1594771200000,
  1594857600000,
  1594944000000,
  1595030400000,
  1595116800000,
  1595203200000,
  1595289600000,
  1595376000000,
  1595462400000,
  1595548800000,
  1595635200000,
  1595721600000,
  1595808000000,
  1595894400000,
  1595980800000,
  1596067200000,
  1596153600000,
  1596240000000,
  1596326400000,
  1596412800000,
  1596499200000,
  1596585600000,
 

In [92]:
open_interest_GS = pd.DataFrame(json.loads(data.text).get('data'))
open_interest_GS['dateList'] = pd.to_datetime(open_interest_GS['dateList'])

In [95]:
data = requests.get('https://fapi.bybt.com/api/grayscale/market/history?symbol=BTC', headers=HEADERS)
historical_data = pd.DataFrame(json.loads(data.text).get('data'))

In [97]:
historical_data.tail(10)

,markerPriceList,dateList,holdingsList,premiumRateList
1976,33.56,1627938000000,37.40,-10.27
1977,31.88,1628024400000,36.16,-11.84
1978,33.14,1628110800000,36.25,-8.58
1979,33.70,1628197200000,36.97,-8.85
1980,36.00,1628283600000,38.93,-7.53
1981,38.55,1628542800000,41.93,-8.06
1982,37.80,1628629200000,42.82,-11.72
1983,38.87,1628715600000,43.24,-10.11
1984,36.86,1628802000000,42.29,-12.84
1985,38.34,1628888400000,42.82,-10.46


# Parse data from
1. https://www.bybt.com/BitcoinOpenInterest

In [98]:
html = get_html('https://www.cftc.gov/dea/futures/financial_lf.htm')
soup = BeautifulSoup(html.text, 'html.parser')
# https://stackoverflow.com/questions/49949849/how-to-parse-the-data-in-pre-tag-using-beautifulsoup

In [99]:
soup

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">

<html lang="en-US" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
<head>
<title>CFTC Commitments of Traders Short Report - Financial Traders in Markets (Futures Only)</title>
<!--begin meta-->
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="0" http-equiv="Expires"/>
<meta content="This is the viewable version of the most recent release of the Financial Traders in Markets short form Futures Only commitments report." name="description">
<meta content="finacial_lf.htm" name="file">
<meta content="commitments, short, CFTC, Finacial Traders in Markets, Futures Only" name="keywords"/>
<meta content="dea" name="office"/>
<!--end meta-->
<script src="http://www.google-analytics.com/ga.js" type="text/javascript"></script><script type="text/javascript">var pageTracker = _gat._getTracker("